# Notebook de cliente

In [1]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer
from preprocess_dataset import remove_empty_entries

import torch

## Setups

In [2]:
CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)

In [3]:
data = "lener_br"
dataset = load_dataset(data)
dataset = remove_empty_entries(dataset)

Reusing dataset lener_br (/home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382)
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-5e59bc59f25f3d7f.arrow
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-8d0457760cd67ee6.arrow
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-74e841c1c151996a.arrow


In [4]:
MAX_LEN = 256
n_labels = 13 #TODO: por enquanto hardcodado 

In [5]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=MAX_LEN, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]

In [6]:
model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

----------------------------------------

## Treinando o modelo

Usando parametros padrão por enquanto só pra ver se ta tudo indo certindo

In [7]:
from trainner import Trainner, create_data_loader
from transformers import AdamW
from torch.utils.data import DataLoader

### Parametros


* batch sizes: 8, 16, 32, 64, 128
* learning rates: 3e-4, 1e-4, 5e-5, 3e-5

Do paper do BERT. Vou colocar num_workers = 4 por enquanto.


Uma melhoria é esconder o data_loader de clientes. Isso é detalhe de implementaçãõ, ou deveria estar na classe NERDataset

In [8]:
optimizer = AdamW(model.parameters(), lr=3e-4, correct_bias=False)

In [9]:
len(pytorch_dataset_train)

7827

In [10]:
loader = DataLoader(pytorch_dataset_train, batch_size=8)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
next(model.parameters()).is_cuda

False

In [13]:
tr = Trainner(device, loader, model, optimizer, max_len=256, num_examples=len(pytorch_dataset_train))

In [14]:
losses, acc = tr._train_epoch()

Iteração 0 -------- Acuracia: 0.0061%------- Loss: 2.5621142387390137 ------ Progresso: 0.00%.
Iteração 100 -------- Acuracia: 8.9494%------- Loss: 0.06855693459510803 ------ Progresso: 10.21%.
Iteração 200 -------- Acuracia: 17.3246%------- Loss: 0.6077423691749573 ------ Progresso: 20.43%.
Iteração 300 -------- Acuracia: 27.1761%------- Loss: 4.609857559204102 ------ Progresso: 30.64%.
Iteração 400 -------- Acuracia: 37.2696%------- Loss: 2.448516845703125 ------ Progresso: 40.86%.
Iteração 500 -------- Acuracia: 46.2173%------- Loss: 1.7864865064620972 ------ Progresso: 51.07%.
Iteração 600 -------- Acuracia: 55.3758%------- Loss: 4.00828218460083 ------ Progresso: 61.29%.
Iteração 700 -------- Acuracia: 64.5783%------- Loss: 2.091496706008911 ------ Progresso: 71.50%.
Iteração 800 -------- Acuracia: 74.3811%------- Loss: 8.647054672241211 ------ Progresso: 81.72%.
Iteração 900 -------- Acuracia: 84.4193%------- Loss: 8.827248573303223 ------ Progresso: 91.93%.


In [15]:
losses

[2.5621142387390137,
 0.6040118932723999,
 5.191826820373535,
 0.12370701879262924,
 0.4006839990615845,
 0.5897175669670105,
 8.018101692199707,
 3.705416202545166,
 0.5099468231201172,
 1.2692986726760864,
 0.7686495780944824,
 0.4089209735393524,
 0.06914522498846054,
 0.48372191190719604,
 1.0620993375778198,
 0.41881632804870605,
 0.7433944344520569,
 0.4222509562969208,
 0.32016754150390625,
 0.6225022673606873,
 2.2197318077087402,
 1.7677035331726074,
 0.11752752959728241,
 3.053004741668701,
 2.759054660797119,
 1.2315773963928223,
 2.217944860458374,
 2.0047292709350586,
 0.9198872447013855,
 3.227958917617798,
 1.6832886934280396,
 0.17047666013240814,
 0.41023001074790955,
 2.0871660709381104,
 4.780689239501953,
 1.665907859802246,
 3.4007809162139893,
 6.38603401184082,
 0.0,
 0.0,
 3.6703460216522217,
 3.2095859050750732,
 5.588028907775879,
 1.7688329219818115,
 1.7484774589538574,
 2.634347915649414,
 1.4856384992599487,
 3.038792133331299,
 4.6449198722839355,
 2.9423

In [16]:
"{:.2f}".format(acc*100) + "%"

ValueError: Unknown format code 'f' for object of type 'str'

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(losses)

In [ ]:
np.mean(losses)

O modelo não está aprendendo, os losses tão completamente aleatórios.